In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [4]:
path = "../data/quickdraw_dataset/"
files = os.listdir(path)

In [5]:
def load_data(files):
    X = []
    y = []
    for i, file in enumerate(files):
        data = np.load(path + file)
        X.append(data)
        labels = np.full((data.shape[0], 1), i)
        y.append(labels)
    X = np.concatenate(X).reshape(-1, 28, 28)
    y = np.concatenate(y)
    #train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return torch.from_numpy(X_train).float(), torch.from_numpy(X_test).float(), torch.from_numpy(y_train).long(), torch.from_numpy(y_test).long()

In [6]:
X_train, X_test, y_train, y_test = load_data(files)

In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([2353080, 28, 28]),
 torch.Size([588271, 28, 28]),
 torch.Size([2353080, 1]),
 torch.Size([588271, 1]))

In [9]:
#dataset and dataloader
class QuickDrawDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = QuickDrawDataset(X_train, y_train)
test_dataset = QuickDrawDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle= False)

In [10]:
#model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [12]:
#model
input_size = X_train.shape[-1]
hidden_size = 128
num_layers = 2
output_size = np.unique(y_train).shape[0]
model = LSTM(input_size, hidden_size, num_layers, output_size).to(device)

In [13]:
#feed a batch through the model
for i, (X, y) in enumerate(train_loader):
    X = X.to(device)
    y = y.to(device)
    print(X.shape)
    out = model(X)
    print(out.shape)
    break

torch.Size([64, 28, 28])
torch.Size([64, 20])


In [14]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
#training
num_epochs = 10
for epoch in range(num_epochs):
    for i, (X, y) in enumerate(train_loader):
        X = X.to(device)
        y = y.to(device)
        #forward
        out = model(X)
        loss = criterion(out, y.squeeze())
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch+1) % 2 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

Epoch 2/10, Loss: 0.3936
Epoch 4/10, Loss: 0.4343
Epoch 6/10, Loss: 0.2324
Epoch 8/10, Loss: 0.2697
Epoch 10/10, Loss: 0.4077


In [16]:
#testing
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        out = model(X)
        _, predictions = torch.max(out, 1)
        n_samples += y.shape[0]
        n_correct += (predictions == y.squeeze()).sum().item()
    acc = 100.0 * n_correct / n_samples
    print(f"Accuracy: {acc:.2f}")

Accuracy: 88.29
